In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

##############################################################
import json
import re

# ✅ JSON 파일 읽기
file_path = "취업_naver_news.json"

with open(file_path, "r", encoding="utf-8") as file:
    news_data = json.load(file)  # JSON 데이터 로드
##############################################################

# ✅ 샘플 데이터셋 (문장 예제)
corpus = [
   "나는 너를 사랑해",
    "나는 코딩을 좋아해",
    "너는 나를 좋아해",
    "너는 파이썬을 공부해",
    "우리는 인공지능을 연구해",
    "딥러닝은 재미있어",
    "파이썬은 강력해",
    "나는 자연어처리를 공부해",
]
##############################################################
# ✅ JSON 데이터에서 'title' 값만 추출하고 한글만 남기기
def extract_korean(text):
    """문장에서 한글만 남기는 함수"""
    return re.sub(r"[^ㄱ-ㅎ가-힣 ]+", " ", text)

news_titles = [extract_korean(item["title"]) for item in news_data if "title" in item]

# ✅ corpus에 한글만 남긴 뉴스 제목 추가
corpus.extend(news_titles)

# ✅ 결과 출력
print("📌 최종 corpus 리스트:")
print(corpus)
##############################################################

# ✅ 단어 사전 만들기 (Tokenization)
word_list = list(set(" ".join(corpus).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
idx_dict = {i: w for w, i in word_dict.items()}

📌 최종 corpus 리스트:
['나는 너를 사랑해', '나는 코딩을 좋아해', '너는 나를 좋아해', '너는 파이썬을 공부해', '우리는 인공지능을 연구해', '딥러닝은 재미있어', '파이썬은 강력해', '나는 자연어처리를 공부해', '경북교육청   학년도 고입전형 분석 결과 발표', '건국대  고용노동부   졸업생 특화 프로그램 사업  선정', '경북지역 고등학교 진학한 타시도 학생 지속적 증가', '고양시  교육발전특구 중부대학교       개소', ' 뷰티대학교육협회  경인여자대학교 뷰티디자인학과와   및   체결', '고양특례시  산업기반 인재양성 본격화', '부안군 근농인재육성재단   년  학기 장학생 신청 접수', ' 손해보험  자립준비청년 기술 교육 지원으로  돌봄  공백 해소', '한수원  상반기 신입사원  명 채용 일 접수 마감', '신용보증기금  우수 인재  신입직원  명  공개 채용', '고양시  중부대학교에  고양      개소', ' 손해보험  자립준비청년 기술 교육  런런챌린지   기 시작', '은행권   일부터 서민금융진흥원 출연금 인상', '교육부  제 회 교육 공공데이터 분석 활용 대회 개최', '원광대  전라북도   지역특성화산업 전문인력양성사업 선정', ' 선우은숙 친언니 강제추행  구속된 유영재  항소심 첫 공판  월  일 진 ', ' 일과 육아  여전히 양자택일 우울한 여성들', '경북 고교  타 시도 학생 유입  천 명  유입  유출 대비  배 많아', '송파새일센터   년   백엔드 개발자 과정 교육생 모집', '괴산군노인복지관  노인 경제자립 도울  취업 알선형 사업 추진', '신용보증기금  상반기 신입직원  명 채용', '  아르바이트 청년  건보료 경감  논의  표심  겨냥', '조선이공대학교   지역맞춤형 일자리 사업  수행기관 선정', '한수원  올해 상반기 신입사원  명 채용 일 접수 마감', '함안군  지역특화형 비자사업 시행 비자요건 완화 우수인재  명 유치', '경북교육청   학년도 고입전형 분석 결과

In [8]:
# ✅ 데이터셋 변환
def make_data(corpus):
    inputs, targets = [], []
    for sentence in corpus:
        words = sentence.split()
        for i in range(len(words) - 1):  # "I love" -> "you"
            x = [word_dict[w] for w in words[:i+1]]
            y = word_dict[words[i+1]]
            inputs.append(x)
            targets.append(y)

    return inputs, targets

inputs, targets = make_data(corpus)

# ✅ 패딩 추가 (문장 길이를 맞춤)
max_len = max(len(seq) for seq in inputs)
inputs_padded = [seq + [0] * (max_len - len(seq)) for seq in inputs]
targets = torch.tensor(targets, dtype=torch.long)

# ✅ 데이터셋 및 DataLoader 생성
class TextDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long)
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

dataset = TextDataset(inputs_padded, targets)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

vocab_size = len(word_dict)  # 단어 개수
embed_size = 10  # 임베딩 차원
hidden_size = 16  # RNN 은닉층 크기
num_classes = len(word_dict)  # 예측할 단어 개수

In [9]:
# ✅ GRU 모델 정의
class GRUTextModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(GRUTextModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)  # 단어 임베딩
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  # 마지막 시점의 GRU 출력을 사용
        return out

# ✅ 모델 생성
model = GRUTextModel(vocab_size, embed_size, hidden_size, num_classes)

# ✅ GPU 사용 가능하면 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [10]:
num_epochs = 100
print("🚀 GRU 모델 학습 시작...")
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# ✅ 모델 저장
model_path = "./gru_news_취업_model.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ 학습된 모델이 저장되었습니다: {model_path}")


🚀 GRU 모델 학습 시작...
Epoch [1/100], Loss: 7.7541
Epoch [2/100], Loss: 6.3901
Epoch [3/100], Loss: 5.5648
Epoch [4/100], Loss: 4.9519
Epoch [5/100], Loss: 4.5075
Epoch [6/100], Loss: 4.1643
Epoch [7/100], Loss: 3.8682
Epoch [8/100], Loss: 3.6813
Epoch [9/100], Loss: 3.4951
Epoch [10/100], Loss: 3.3519
Epoch [11/100], Loss: 3.2314
Epoch [12/100], Loss: 3.1272
Epoch [13/100], Loss: 3.0204
Epoch [14/100], Loss: 2.9848
Epoch [15/100], Loss: 2.9185
Epoch [16/100], Loss: 2.8396
Epoch [17/100], Loss: 2.7973
Epoch [18/100], Loss: 2.7400
Epoch [19/100], Loss: 2.7239
Epoch [20/100], Loss: 2.6626
Epoch [21/100], Loss: 2.6167
Epoch [22/100], Loss: 2.5701
Epoch [23/100], Loss: 2.5675
Epoch [24/100], Loss: 2.5432
Epoch [25/100], Loss: 2.5016
Epoch [26/100], Loss: 2.4890
Epoch [27/100], Loss: 2.4321
Epoch [28/100], Loss: 2.4373
Epoch [29/100], Loss: 2.4183
Epoch [30/100], Loss: 2.4000
Epoch [31/100], Loss: 2.4260
Epoch [32/100], Loss: 2.3954
Epoch [33/100], Loss: 2.3564
Epoch [34/100], Loss: 2.3454
Epoch

In [11]:
model_path = "./gru_news_취업_model.pth"
# ✅ 저장된 RNN 모델 불러오기
def load_model(model_path, vocab_size, embed_size, hidden_size, num_classes):
    model = GRUTextModel(vocab_size, embed_size, hidden_size, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
    model.eval()
    return model

# ✅ 모델 불러오기
loaded_model = load_model(model_path, vocab_size, embed_size, hidden_size, num_classes)
print("✅ 모델이 성공적으로 불러와졌습니다!")


✅ 모델이 성공적으로 불러와졌습니다!


In [12]:

import torch.nn.functional as F

def predict_next_word(model, sentence):
    """
    저장된 RNN 모델을 사용하여 주어진 문장의 다음 단어를 예측하는 함수.
    """
    # ✅ 입력 문장을 정수 인코딩
    words = sentence.split()
    input_seq = [word_dict[w] for w in words if w in word_dict]

    # ✅ 패딩 추가 (길이를 맞추기 위해)
    input_padded = input_seq + [0] * (max_len - len(input_seq))
    input_tensor = torch.tensor([input_padded], dtype=torch.long)

    # ✅ 모델 예측
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output[0], dim=0)
        predicted_idx = torch.argmax(probabilities).item()
        confidence = probabilities[predicted_idx].item()

    predicted_word = idx_dict[predicted_idx]

    print(f"🔍 입력 문장: '{sentence}'")
    print(f"📊 예측된 단어: '{predicted_word}'")
    print(f"✅ 예측 확률: {confidence * 100:.2f}%")

# 🏆 샘플 문장 예측 실행
sample_sentence = "취업  "
predict_next_word(loaded_model, sample_sentence)

🔍 입력 문장: '취업  '
📊 예측된 단어: '사이트'
✅ 예측 확률: 12.66%
